In [1]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objs as go
import random
import itertools
from itertools import product
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_squared_error
import pmdarima as pm
from pmdarima.arima import auto_arima
import warnings

In [2]:
%cd /data/p_dsi/teams2023/team7

/gpfs52/data/p_dsi/teams2023/team7


In [3]:
with open("dataframes_units_per_article_monthly.json") as json_file:
    data = json.load(json_file)

In [4]:
revert_dict = {}
for k in data.keys():
    temp = pd.DataFrame.from_dict(data[k])
    temp["DATE"] = pd.to_datetime(temp["DATE"])
    temp = temp.set_index("DATE")
    revert_dict.update({f"{k}": temp})

In [5]:
short_list = random.sample(list(revert_dict.keys()),5)
sample_dict = {k:revert_dict[k] for k in short_list}
print(sample_dict)

{'23240':             UNITS
DATE             
2016-03-31    671
2016-04-30   1767
2016-05-31   3040
2016-06-30   3125
2016-07-31   3160
2016-08-31   2210
2016-09-30   2435
2016-10-31   2681
2016-11-30   4272
2016-12-31   4409
2017-01-31   2870
2017-02-28   2924
2017-03-31   2627
2017-04-30   2359
2017-05-31   3411
2017-06-30   3650
2017-07-31   3056
2017-08-31   3604
2017-09-30   3437
2017-10-31   4052
2017-11-30   4012
2017-12-31   3260
2018-01-31   3783
2018-02-28   4130
2018-03-31   5925
2018-04-30   3750
2018-05-31   4582
2018-06-30   4429
2018-07-31   3683
2018-08-31   3082
2018-09-30   3083
2018-10-31   4111, '1733':             UNITS
DATE             
2015-04-30    364
2015-05-31    421
2015-06-30    576
2015-07-31    677
2015-08-31   1187
2015-09-30   1273
2015-10-31    988
2015-11-30   1034
2015-12-31    768
2016-01-31    640
2016-02-29    669
2016-03-31    975
2016-04-30    804
2016-05-31   1053
2016-06-30   1565
2016-07-31   1291
2016-08-31   1662
2016-09-30   1552
2016-10-3

## Funciton definition

In [6]:
## no recursion
def take_log_auto_d(train):

    stepwise_fit = pm.auto_arima(train, start_p=1, start_q=1,
                             max_p=3, max_q=3, m=12,
                             start_P=0, seasonal=True,
                             start_d=0, max_d = 2,
                             D = 1, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise
    
    if stepwise_fit.order == (0,1,0) or stepwise_fit.order == (0,0,0):
        return stepwise_fit, "Not stationary after log"
        
    else:
        return stepwise_fit, 1

## NO recursion with fit & prediction 

### Sample test

In [7]:
## NO RECURSION WITH FIT / PREDCIT
best_models_f1 = {}
no_transformation_f1 = {}
first_log_f1 = {}
non_stationary_f1 = {}
sep_pred_dict = {}
oct_pred_dict = {}
nov_pred_dict = {}
error_article_id_f1 = []
count = 0
error_msg = {}

for article_id, data in sample_dict.items():
    count += 1
    print(count)
    train = data[:len(data) - 2]
    test = data[len(data) - 2:len(data)]
    
    print(len(train),len(data)-1)
    
    try:
        stepwise_fit = pm.auto_arima(train, start_p=1, start_q=1,
                             max_p=3, max_q=3, m=12,
                             seasonal=True,
                             start_d=0, max_d = 2, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise

        
        if stepwise_fit.order == (0,1,0) or stepwise_fit.order == (0,0,0):
            data_log = np.log(data)
            data_log = np.where(np.isinf(data_log), 0, data_log)
            data_log = pd.DataFrame(data_log, columns=data.columns, index=data.index)
      
            train_log = data_log[:len(data_log) - 2]
            test_log = data_log[len(data_log) - 2:len(data_log)]
                      
            
            model, order = take_log_auto_d(train_log)
            best_models_f1[article_id] = model
            
            if order == 1:
                first_log_f1[article_id] = model
            elif order == "Not stationary after log":
                non_stationary_f1[article_id] = model
            else: print("log_order = ", order)
                
       
            stepwise_fit.fit(train_log)
            sep_oct_forecast = stepwise_fit.predict(n_periods=2)
            sep_oct_forecast = np.exp(sep_oct_forecast)

            sep_pred_dict[article_id] = sep_oct_forecast[0]
            oct_pred_dict[article_id] = sep_oct_forecast[1]            
            
            stepwise_fit.fit(data_log)
            nov_forecast = stepwise_fit.predict(n_periods=1) 
            nov_forecast = np.exp(nov_forecast)
            nov_pred_dict[article_id] = nov_forecast[0]   
            
        
            #future_forecast = save some how (data frame)
        
        else:
            best_models_f1[article_id] = stepwise_fit
            no_transformation_f1[article_id] = stepwise_fit
            
            stepwise_fit.fit(train)
            sep_oct_forecast = stepwise_fit.predict(n_periods=2)
            
            sep_pred_dict[article_id] = sep_oct_forecast[0]
            oct_pred_dict[article_id] = sep_oct_forecast[1]      
        
        
            stepwise_fit.fit(data)
            nov_forecast = stepwise_fit.predict(n_periods=1)
            nov_pred_dict[article_id] = nov_forecast[0]  
       
        
    except Exception as e:
        print(f"Failed to fit a model for article {article_id}: {e}")
        error_msg[article_id] = e
        error_article_id_f1.append(article_id)
        continue

1
30 31
Performing stepwise search to minimize aic
 ARIMA(1,1,1)(1,0,1)[12] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=478.263, Time=0.01 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=481.340, Time=0.02 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=480.848, Time=0.07 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=476.533, Time=0.01 sec
 ARIMA(0,1,0)(1,0,0)[12] intercept   : AIC=480.241, Time=0.02 sec
 ARIMA(0,1,0)(0,0,1)[12] intercept   : AIC=480.239, Time=0.02 sec
 ARIMA(0,1,0)(1,0,1)[12] intercept   : AIC=482.237, Time=0.07 sec
 ARIMA(1,1,0)(0,0,0)[12] intercept   : AIC=479.375, Time=0.01 sec
 ARIMA(0,1,1)(0,0,0)[12] intercept   : AIC=478.848, Time=0.04 sec
 ARIMA(1,1,1)(0,0,0)[12] intercept   : AIC=inf, Time=0.09 sec

Best model:  ARIMA(0,1,0)(0,0,0)[12]          
Total fit time: 0.630 seconds
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=0.32 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=18.57

/home/sohnm/.local/lib/python3.9/site-packages/pmdarima/arima/utils.py:117: UserWarning: Appropriate D value may not have been reached; length of seasonally-differenced array (7) is shorter than m (12). Using D=1
  warnings.warn("Appropriate D value may not have been reached; "


 ARIMA(1,0,1)(1,1,1)[12] intercept   : AIC=inf, Time=0.31 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=103.820, Time=0.01 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=inf, Time=0.19 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=0.08 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=116.252, Time=0.01 sec
 ARIMA(0,0,0)(1,1,0)[12] intercept   : AIC=105.816, Time=0.02 sec
 ARIMA(0,0,0)(0,1,1)[12] intercept   : AIC=inf, Time=0.10 sec
 ARIMA(0,0,0)(1,1,1)[12] intercept   : AIC=inf, Time=0.21 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=105.057, Time=0.05 sec
 ARIMA(0,0,1)(0,1,0)[12] intercept   : AIC=inf, Time=0.03 sec
 ARIMA(1,0,1)(0,1,0)[12] intercept   : AIC=inf, Time=0.09 sec

Best model:  ARIMA(0,0,0)(0,1,0)[12] intercept
Total fit time: 1.103 seconds
Performing stepwise search to minimize aic


/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/pandas/core/internals/blocks.py:402: RuntimeWarning: divide by zero encountered in log
  result = func(self.values, **kwargs)


 ARIMA(1,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=0.16 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=37.667, Time=0.01 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=40.355, Time=0.22 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=39.110, Time=0.06 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=41.934, Time=0.01 sec
 ARIMA(0,0,0)(1,1,0)[12] intercept   : AIC=39.667, Time=0.02 sec
 ARIMA(0,0,0)(0,1,1)[12] intercept   : AIC=39.667, Time=0.01 sec
 ARIMA(0,0,0)(1,1,1)[12] intercept   : AIC=41.667, Time=0.02 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=38.360, Time=0.02 sec
 ARIMA(0,0,1)(0,1,0)[12] intercept   : AIC=37.110, Time=0.03 sec


/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(0,0,1)(1,1,0)[12] intercept   : AIC=39.110, Time=0.06 sec
 ARIMA(0,0,1)(1,1,1)[12] intercept   : AIC=41.110, Time=0.09 sec
 ARIMA(1,0,1)(0,1,0)[12] intercept   : AIC=inf, Time=0.07 sec
 ARIMA(0,0,2)(0,1,0)[12] intercept   : AIC=inf, Time=0.09 sec
 ARIMA(1,0,2)(0,1,0)[12] intercept   : AIC=inf, Time=0.13 sec
 ARIMA(0,0,1)(0,1,0)[12]             : AIC=inf, Time=0.03 sec

Best model:  ARIMA(0,0,1)(0,1,0)[12] intercept
Total fit time: 1.041 seconds


In [8]:
print(sep_pred_dict)
print('================')
print(oct_pred_dict)
print('================')
print(nov_pred_dict)

{'23240': 3082.000000000002, '1733': 2976.521242476833, '15114': 1544.8845006229933, '7400': 1836.5462730955085}
{'23240': 3082.000000000002, '1733': 3039.323905347408, '15114': 1649.4084669367837, '7400': 6063.118518027638}
{'23240': 4111.000000000001, '1733': 3008.3942076740145, '15114': 1755.698699630453, '7400': 8041.42729322895}


In [9]:
## NO RECURSION WITH FIT / PREDCIT
best_models_f1 = {}
no_transformation_f1 = {}
first_log_f1 = {}
non_stationary_f1 = {}
sep_pred_dict = {}
oct_pred_dict = {}
nov_pred_dict = {}
error_article_id_f1 = []
count = 0
error_msg = {}

for article_id, data in revert_dict.items():
    count += 1
    print(count)
    train = data[:len(data) - 2]
    test = data[len(data) - 2:len(data)]
    
    print(len(train),len(data)-1)
    
    try:
        stepwise_fit = pm.auto_arima(train, start_p=1, start_q=1,
                             max_p=3, max_q=3, m=12,
                             seasonal=True,
                             start_d=0, max_d = 2, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise

        
        if stepwise_fit.order == (0,1,0) or stepwise_fit.order == (0,0,0):
            data_log = np.log(data)
            data_log = np.where(np.isinf(data_log), 0, data_log)
            data_log = pd.DataFrame(data_log, columns=data.columns, index=data.index)
      
            train_log = data_log[:len(data_log) - 2]
            test_log = data_log[len(data_log) - 2:len(data_log)]
                      
            
            model, order = take_log_auto_d(train_log)
            best_models_f1[article_id] = model
            
            if order == 1:
                first_log_f1[article_id] = model
            elif order == "Not stationary after log":
                non_stationary_f1[article_id] = model
            else: print("log_order = ", order)
                
       
            stepwise_fit.fit(train_log)
            sep_oct_forecast = stepwise_fit.predict(n_periods=2)
            sep_oct_forecast = np.exp(sep_oct_forecast)

            sep_pred_dict[article_id] = sep_oct_forecast[0]
            oct_pred_dict[article_id] = sep_oct_forecast[1]            
            
            stepwise_fit.fit(data_log)
            nov_forecast = stepwise_fit.predict(n_periods=1) 
            nov_forecast = np.exp(nov_forecast)
            nov_pred_dict[article_id] = nov_forecast[0]   
            
        
            #future_forecast = save some how (data frame)
        
        else:
            best_models_f1[article_id] = stepwise_fit
            no_transformation_f1[article_id] = stepwise_fit
            
            stepwise_fit.fit(train)
            sep_oct_forecast = stepwise_fit.predict(n_periods=2)
            
            sep_pred_dict[article_id] = sep_oct_forecast[0]
            oct_pred_dict[article_id] = sep_oct_forecast[1]      
        
        
            stepwise_fit.fit(data)
            nov_forecast = stepwise_fit.predict(n_periods=1)
            nov_pred_dict[article_id] = nov_forecast[0]  
       
        
    except Exception as e:
        print(f"Failed to fit a model for article {article_id}: {e}")
        error_msg[article_id] = e
        error_article_id_f1.append(article_id)
        continue

1
41 42
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(1,0,1)[12] intercept   : AIC=603.864, Time=0.21 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=610.219, Time=0.01 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=604.551, Time=0.04 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=600.211, Time=0.07 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=782.314, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[12] intercept   : AIC=599.680, Time=0.03 sec
 ARIMA(0,0,1)(1,0,0)[12] intercept   : AIC=601.726, Time=0.04 sec
 ARIMA(0,0,1)(1,0,1)[12] intercept   : AIC=602.210, Time=0.22 sec
 ARIMA(1,0,1)(0,0,0)[12] intercept   : AIC=603.581, Time=0.02 sec
 ARIMA(0,0,2)(0,0,0)[12] intercept   : AIC=601.677, Time=0.05 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=604.251, Time=0.01 sec
 ARIMA(1,0,2)(0,0,0)[12] intercept   : AIC=603.682, Time=0.03 sec
 ARIMA(0,0,1)(0,0,0)[12]             : AIC=inf, Time=0.02 sec

Best model:  ARIMA(0,0,1)(0,0,0)[12] intercept
Total fit time: 0.766 seconds
2
41 42
Performin

 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=0.23 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=301.120, Time=0.01 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=289.851, Time=0.05 sec
 ARIMA(1,0,0)(2,1,0)[12] intercept   : AIC=284.903, Time=0.37 sec
 ARIMA(1,0,0)(1,1,1)[12] intercept   : AIC=inf, Time=0.29 sec
 ARIMA(1,0,0)(0,1,1)[12] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(1,0,0)(2,1,1)[12] intercept   : AIC=286.883, Time=0.40 sec
 ARIMA(0,0,0)(1,1,0)[12] intercept   : AIC=288.550, Time=0.13 sec
 ARIMA(2,0,0)(1,1,0)[12] intercept   : AIC=284.780, Time=0.26 sec
 ARIMA(1,0,1)(1,1,0)[12] intercept   : AIC=284.764, Time=0.26 sec
 ARIMA(0,0,1)(1,1,0)[12] intercept   : AIC=inf, Time=0.17 sec
 ARIMA(2,0,1)(1,1,0)[12] intercept   : AIC=286.747, Time=0.28 sec
 ARIMA(1,0,0)(1,1,0)[12]             : AIC=283.174, Time=0.10 sec

Best model:  ARIMA(1,0,0)(1,1,0)[12] intercept
Total fit time: 3.362 seconds
7
41 42
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(1,0,1)[12] inte

12
41 42
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(1,0,1)[12] intercept   : AIC=inf, Time=0.26 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=621.738, Time=0.01 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=580.047, Time=0.12 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=604.082, Time=0.09 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=734.441, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=578.052, Time=0.05 sec
 ARIMA(1,0,0)(0,0,1)[12] intercept   : AIC=580.047, Time=0.13 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=inf, Time=0.15 sec
 ARIMA(2,0,0)(0,0,0)[12] intercept   : AIC=579.994, Time=0.11 sec
 ARIMA(1,0,1)(0,0,0)[12] intercept   : AIC=580.018, Time=0.09 sec
 ARIMA(0,0,1)(0,0,0)[12] intercept   : AIC=602.084, Time=0.03 sec
 ARIMA(2,0,1)(0,0,0)[12] intercept   : AIC=581.416, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[12]             : AIC=578.464, Time=0.01 sec

Best model:  ARIMA(1,0,0)(0,0,0)[12] intercept
Total fit time: 1.097 seconds
13
41 42
Performing 

 ARIMA(1,0,1)(0,0,0)[12] intercept   : AIC=745.075, Time=0.07 sec
 ARIMA(1,0,1)(1,0,0)[12] intercept   : AIC=747.028, Time=0.05 sec
 ARIMA(0,0,1)(0,0,0)[12] intercept   : AIC=781.253, Time=0.04 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=749.236, Time=0.01 sec
 ARIMA(2,0,1)(0,0,0)[12] intercept   : AIC=745.519, Time=0.11 sec
 ARIMA(1,0,2)(0,0,0)[12] intercept   : AIC=743.248, Time=0.05 sec
 ARIMA(1,0,2)(1,0,0)[12] intercept   : AIC=745.244, Time=0.05 sec
 ARIMA(1,0,2)(0,0,1)[12] intercept   : AIC=745.319, Time=0.05 sec
 ARIMA(1,0,2)(1,0,1)[12] intercept   : AIC=747.178, Time=0.10 sec
 ARIMA(0,0,2)(0,0,0)[12] intercept   : AIC=753.623, Time=0.06 sec
 ARIMA(2,0,2)(0,0,0)[12] intercept   : AIC=743.573, Time=0.09 sec
 ARIMA(1,0,3)(0,0,0)[12] intercept   : AIC=744.120, Time=0.08 sec
 ARIMA(0,0,3)(0,0,0)[12] intercept   : AIC=750.788, Time=0.07 sec
 ARIMA(2,0,3)(0,0,0)[12] intercept   : AIC=745.475, Time=0.10 sec
 ARIMA(1,0,2)(0,0,0)[12]             : AIC=744.452, Time=0.06 sec

Best mode

 ARIMA(0,1,1)(0,1,1)[12]             : AIC=-69.317, Time=0.28 sec
 ARIMA(0,1,1)(0,1,0)[12]             : AIC=-69.361, Time=0.06 sec
 ARIMA(0,1,1)(1,1,0)[12]             : AIC=-69.040, Time=0.13 sec
 ARIMA(0,1,1)(1,1,1)[12]             : AIC=-67.334, Time=0.30 sec
 ARIMA(1,1,1)(0,1,0)[12]             : AIC=-63.993, Time=0.07 sec
 ARIMA(0,1,2)(0,1,0)[12]             : AIC=-67.854, Time=0.10 sec
 ARIMA(1,1,0)(0,1,0)[12]             : AIC=-67.906, Time=0.06 sec
 ARIMA(1,1,2)(0,1,0)[12]             : AIC=-65.854, Time=0.10 sec
 ARIMA(0,1,1)(0,1,0)[12] intercept   : AIC=-70.703, Time=0.06 sec
 ARIMA(0,1,1)(1,1,0)[12] intercept   : AIC=-73.109, Time=0.31 sec
 ARIMA(0,1,1)(2,1,0)[12] intercept   : AIC=-71.973, Time=0.71 sec
 ARIMA(0,1,1)(1,1,1)[12] intercept   : AIC=-71.955, Time=0.50 sec
 ARIMA(0,1,1)(0,1,1)[12] intercept   : AIC=inf, Time=0.19 sec
 ARIMA(0,1,1)(2,1,1)[12] intercept   : AIC=-69.972, Time=0.74 sec
 ARIMA(0,1,0)(1,1,0)[12] intercept   : AIC=-66.335, Time=0.15 sec
 ARIMA(1,1,1)(

...

 ARIMA(1,0,1)(0,1,1)[12] intercept   : AIC=inf, Time=0.16 sec
 ARIMA(0,0,0)(0,1,0)[12] intercept   : AIC=37.667, Time=0.01 sec
 ARIMA(1,0,0)(1,1,0)[12] intercept   : AIC=40.355, Time=0.22 sec
 ARIMA(0,0,1)(0,1,1)[12] intercept   : AIC=39.110, Time=0.06 sec
 ARIMA(0,0,0)(0,1,0)[12]             : AIC=41.934, Time=0.01 sec
 ARIMA(0,0,0)(1,1,0)[12] intercept   : AIC=39.667, Time=0.01 sec
 ARIMA(0,0,0)(0,1,1)[12] intercept   : AIC=39.667, Time=0.01 sec
 ARIMA(0,0,0)(1,1,1)[12] intercept   : AIC=41.667, Time=0.02 sec
 ARIMA(1,0,0)(0,1,0)[12] intercept   : AIC=38.360, Time=0.02 sec
 ARIMA(0,0,1)(0,1,0)[12] intercept   : AIC=37.110, Time=0.03 sec


/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1899: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ar)**-1
/accre/arch/easybuild/software/BinDist/Anaconda3/2022.05/lib/python3.9/site-packages/statsmodels/tsa/statespace/sarimax.py:1906: RuntimeWarning: divide by zero encountered in reciprocal
  return np.roots(self.polynomial_reduced_ma)**-1


 ARIMA(0,0,1)(1,1,0)[12] intercept   : AIC=39.110, Time=0.06 sec
 ARIMA(0,0,1)(1,1,1)[12] intercept   : AIC=41.110, Time=0.08 sec
 ARIMA(1,0,1)(0,1,0)[12] intercept   : AIC=inf, Time=0.07 sec
 ARIMA(0,0,2)(0,1,0)[12] intercept   : AIC=inf, Time=0.09 sec
 ARIMA(1,0,2)(0,1,0)[12] intercept   : AIC=inf, Time=0.13 sec
 ARIMA(0,0,1)(0,1,0)[12]             : AIC=inf, Time=0.03 sec

Best model:  ARIMA(0,0,1)(0,1,0)[12] intercept
Total fit time: 1.028 seconds
412
41 42
Performing stepwise search to minimize aic
 ARIMA(1,0,1)(1,0,1)[12] intercept   : AIC=570.265, Time=0.19 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=580.831, Time=0.01 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=566.511, Time=0.03 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=571.896, Time=0.07 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=665.523, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=564.745, Time=0.02 sec
 ARIMA(1,0,0)(0,0,1)[12] intercept   : AIC=566.377, Time=0.03 sec
 ARIMA(1,0,0)(1,0,1)[12] inte

 ARIMA(1,0,1)(1,0,1)[12] intercept   : AIC=380.453, Time=0.19 sec
 ARIMA(0,0,0)(0,0,0)[12] intercept   : AIC=399.438, Time=0.01 sec
 ARIMA(1,0,0)(1,0,0)[12] intercept   : AIC=376.672, Time=0.09 sec
 ARIMA(0,0,1)(0,0,1)[12] intercept   : AIC=inf, Time=0.15 sec
 ARIMA(0,0,0)(0,0,0)[12]             : AIC=456.057, Time=0.01 sec
 ARIMA(1,0,0)(0,0,0)[12] intercept   : AIC=376.992, Time=0.04 sec
 ARIMA(1,0,0)(2,0,0)[12] intercept   : AIC=378.196, Time=0.22 sec
 ARIMA(1,0,0)(1,0,1)[12] intercept   : AIC=378.069, Time=0.18 sec
 ARIMA(1,0,0)(0,0,1)[12] intercept   : AIC=376.081, Time=0.10 sec
 ARIMA(1,0,0)(0,0,2)[12] intercept   : AIC=inf, Time=0.25 sec
 ARIMA(1,0,0)(1,0,2)[12] intercept   : AIC=380.068, Time=0.23 sec
 ARIMA(0,0,0)(0,0,1)[12] intercept   : AIC=inf, Time=0.07 sec
 ARIMA(2,0,0)(0,0,1)[12] intercept   : AIC=377.754, Time=0.16 sec
 ARIMA(1,0,1)(0,0,1)[12] intercept   : AIC=377.946, Time=0.17 sec
 ARIMA(2,0,1)(0,0,1)[12] intercept   : AIC=inf, Time=0.20 sec
 ARIMA(1,0,0)(0,0,1)[12]  

 ARIMA(0,1,1)(1,0,1)[12]             : AIC=inf, Time=0.12 sec
 ARIMA(1,1,1)(0,0,0)[12]             : AIC=507.107, Time=0.04 sec
 ARIMA(0,1,2)(0,0,0)[12]             : AIC=507.921, Time=0.04 sec
 ARIMA(1,1,0)(0,0,0)[12]             : AIC=506.425, Time=0.01 sec
 ARIMA(1,1,2)(0,0,0)[12]             : AIC=508.970, Time=0.05 sec

Best model:  ARIMA(0,1,1)(0,0,0)[12]          
Total fit time: 1.660 seconds
430
6 7
Failed to fit a model for article 33: There are no more samples after a first-order seasonal differencing. See http://alkaline-ml.com/pmdarima/seasonal-differencing-issues.html for a more in-depth explanation and potential work-arounds.
431
13 14
Failed to fit a model for article 3042: zero-size array to reduction operation maximum which has no identity
432
30 31
Failed to fit a model for article 12275: All lag values up to 'maxlag' produced singular matrices. Consider using a longer series, a different lag term or a different test.
433
41 42
Performing stepwise search to minimize 

In [10]:
# no_transformation_f1
print(len(no_transformation_f1))

239


In [11]:
# first_log_f1
print(len(first_log_f1))

80


In [12]:
first_log_f1

{'97759': ARIMA(order=(0, 0, 1), scoring_args={}, seasonal_order=(0, 1, 1, 12),
       suppress_warnings=True),
 '15505': ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(0, 1, 0, 12),
       suppress_warnings=True, with_intercept=False),
 '136043': ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(2, 1, 1, 12),
       suppress_warnings=True),
 '114572': ARIMA(order=(3, 0, 1), scoring_args={}, seasonal_order=(2, 1, 0, 12),
       suppress_warnings=True),
 '7099618': ARIMA(order=(0, 1, 1), scoring_args={}, seasonal_order=(1, 1, 0, 12),
       suppress_warnings=True),
 '122817': ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(1, 1, 0, 12),
       suppress_warnings=True, with_intercept=False),
 '831': ARIMA(order=(0, 0, 1), scoring_args={}, seasonal_order=(1, 1, 0, 12),
       suppress_warnings=True),
 '122630': ARIMA(order=(1, 0, 0), scoring_args={}, seasonal_order=(1, 1, 0, 12),
       suppress_warnings=True),
 '2832': ARIMA(order=(1, 0, 0), scoring_args={}, seasona

In [13]:
# non_stationary_f1
print(len(non_stationary_f1))

25


In [14]:
print(len(error_article_id_f1))

92


In [15]:
print(error_article_id_f1)

['1408', '15454', '15539', '15046', '14876', '14995', '15352', '1814', '15556', '15369', '15641', '15658', '15386', '1427', '1400', '3561', '15012', '1411', '1421', '4042', '3051', '4012', '4070', '184', '181', '1435', '3065', '4022', '3024', '182', '6258', '4058', '1404', '2360', '6251', '6252', '4020', '4034', '189', '6254', '4046', '3036', '3816', '4040', '6253', '14808', '4054', '4028', '6505', '4915', '3068', '4002', '3019', '6699', '3767', '122', '113', '12394', '185', '6249', '1414', '1416', '1402', '6255', '3034', '1432', '6256', '4068', '4062', '4032', '4030', '4044', '3351', '15216', '14978', '4050', '4014', '118', '7788', '3040', '3812', '4018', '187', '3030', '4263', '3058', '6241', '6128', '2928', '33', '3042', '12275']


In [16]:
print(len(sep_pred_dict))

343


In [17]:
print(len(oct_pred_dict))

343


In [18]:
print(len(nov_pred_dict))

343


## Saving the errors

In [19]:
error_article_id_f1
len(error_article_id_f1)

92

In [20]:
error_msg
error_msg = {key: str(value) for key, value in error_msg.items()}
print(len(error_msg))

92


In [21]:
with open("error_msg.json_v2", "w") as file:
    json.dump(error_msg, file)



In [22]:
with open("error_msg.json") as json_file:
    error_msg = json.load(json_file)

error_msg

{'1408': 'zero-size array to reduction operation maximum which has no identity',
 '15454': "All lag values up to 'maxlag' produced singular matrices. Consider using a longer series, a different lag term or a different test.",
 '15539': "All lag values up to 'maxlag' produced singular matrices. Consider using a longer series, a different lag term or a different test.",
 '15046': "All lag values up to 'maxlag' produced singular matrices. Consider using a longer series, a different lag term or a different test.",
 '14876': "All lag values up to 'maxlag' produced singular matrices. Consider using a longer series, a different lag term or a different test.",
 '14995': "All lag values up to 'maxlag' produced singular matrices. Consider using a longer series, a different lag term or a different test.",
 '15352': "All lag values up to 'maxlag' produced singular matrices. Consider using a longer series, a different lag term or a different test.",
 '1814': "All lag values up to 'maxlag' produced 

## Saving the prediction values for each ariticle_id in json

In [23]:
# Write new_dict to a JSON file
with open("sep_prediction_v2.json", "w") as file:
    json.dump(sep_pred_dict, file)
    
with open("oct_prediction_v2.json", "w") as file:
    json.dump(oct_pred_dict, file)
    
with open("nov_prediction_v2.json", "w") as file:
    json.dump(nov_pred_dict, file)

In [24]:
with open("nov_prediction_v2.json") as json_file:
    nov_pred = json.load(json_file)

nov_pred
print(len(nov_pred))

343


In [25]:
nov_pred

{'97980': 3182.0242257899513,
 '97793': 2227.0143164633496,
 '97759': 5652.000000000003,
 '15505': 5999.387861728035,
 '15284': 2825.610537484665,
 '7099616': 2521.21396195319,
 '26784': 3086.544674982978,
 '136043': 1315.9713933070614,
 '123888': 8670.358292364832,
 '1374': 1171.9026456294835,
 '146107': 106.00544758434279,
 '826': 5158.311322200563,
 '114674': 9344.214987607866,
 '136094': 5503.063594977977,
 '95872': 1140.0531179622988,
 '114572': 4125.000000000004,
 '85876': 421.9271151119468,
 '3429': 5549.146980090516,
 '114589': 3957.8839142209854,
 '136128': 2009.4466963252194,
 '3945': 2915.7543954103876,
 '97725': 4449.407670490662,
 '7099618': 8388.207790498036,
 '122817': 54.00000000000001,
 '831': 3004.9999999999977,
 '122630': 4014.000000000003,
 '106497': 1414.0568683437468,
 '830': 2279.63045139853,
 '11680': 411.3521139180809,
 '189701': 577.9096498133243,
 '97946': 11230.957109957464,
 '2832': 3964.999999999999,
 '3431': 6624.67708962112,
 '123905': 3763.743678858188,

## Handling erroneous articles with 2 month average

In [26]:
nov_error_pred = {}
for aritcle_id in error_article_id_f1:
    data = revert_dict[aritcle_id]
    two_month = data[-2: ]
    nov_predict = np.mean(two_month, axis = 0)
    nov_error_pred[aritcle_id] = nov_predict


    
nov_error_pred = {key: float(value) for key, value in nov_error_pred.items()}

In [27]:
print(len(nov_error_pred))

92


In [28]:
print(len(nov_pred))

343


In [29]:
nov_final_pred_2m = dict(nov_pred, **nov_error_pred)
nov_final_pred_2m
print(len(nov_final_pred_2m))

435


In [30]:
import csv 

#nov_final_pred_2m

with open('nov_final_pred_2m_final.csv', 'w', newline='') as file:
    # Create a CSV writer object
    writer = csv.writer(file)

    # Write the header row
    writer.writerow(['article_id', 'UNITS'])

    # Write the data rows
    for key, val in nov_final_pred_2m.items():
        writer.writerow([key, val])

In [31]:
## open
df = pd.read_csv('nov_final_pred_2m_final.csv')
df

,article_id,UNITS
0,97980,3182.024226
1,97793,2227.014316
2,97759,5652.000000
3,15505,5999.387862
4,15284,2825.610537
...,...,...
430,6128,851.000000
431,2928,306.000000
432,33,1228.000000
433,3042,824.500000
